In [23]:
#
# Empaquetado del entrenamiento del modelo
#
def train_estimator(alpha=0.5, l1_ratio=0.5, verbose=1):

    import os
    import pickle

    import pandas as pd
    from sklearn.linear_model import ElasticNet
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    from sklearn.model_selection import train_test_split

    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    df = pd.read_csv(url, sep=";")

    y = df["quality"]
    x = df.copy()
    x.pop("quality")

    (x_train, x_test, y_train, y_test) = train_test_split(
        x,
        y,
        test_size=0.25,
        random_state=0,
    )

    estimator = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=12345)

    estimator.fit(x_train, y_train)
    y_pred = estimator.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    if verbose > 0:
        print(estimator, ":", sep="")
        print(f"  MSE: {mse}")
        print(f"  MAE: {mae}")
        print(f"  R2: {r2}")

    if not os.path.exists("estimator.pickle"):
        saved_estimator = None
    else:
        with open("estimator.pickle", "rb") as file:
            saved_estimator = pickle.load(file)

    if saved_estimator is None or estimator.score(
        x_test, y_test
    ) > saved_estimator.score(x_test, y_test):
        with open("estimator.pickle", "wb") as file:
            pickle.dump(estimator, file)


In [24]:
train_estimator(alpha=0.5, l1_ratio=0.5, verbose=1)
print("Modelo entrenado y guardado en 'estimator.pickle'.")
print("Para predecir, use el comando 'predict.py'.")

ElasticNet(alpha=0.5, random_state=12345):
  MSE: 0.5294843132862007
  MAE: 0.5894666734018875
  R2: 0.13368827268570616
Modelo entrenado y guardado en 'estimator.pickle'.
Para predecir, use el comando 'predict.py'.


In [25]:
train_estimator(alpha=0.2, l1_ratio=0.9, verbose=1)
print("Modelo entrenado y guardado en 'estimator.pickle'.")
print("Para predecir, use el comando 'predict.py'.")

ElasticNet(alpha=0.2, l1_ratio=0.9, random_state=12345):
  MSE: 0.4838276670487889
  MAE: 0.5560295747909655
  R2: 0.20838904676497616
Modelo entrenado y guardado en 'estimator.pickle'.
Para predecir, use el comando 'predict.py'.


In [26]:
for i in range(5):
  for j in range(5):
    train_estimator(alpha=0.1 + i * 0.1, l1_ratio=0.1 + j * 0.1, verbose=0)
    

In [27]:
#
# Uso del modelo en productivo
#
def use_estimator():

    import pandas as pd
    import pickle

    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    df = pd.read_csv(url, sep=";")

    y = df["quality"]
    x = df.copy()
    x.pop("quality")

    with open("estimator.pickle", "rb") as file:
        estimator = pickle.load(file)

    y_pred = estimator.predict(x)

    return y_pred

use_estimator()


array([5.24090703, 5.31521455, 5.33699336, ..., 5.80575678, 5.50547416,
       5.78406716], shape=(1599,))

In [28]:
# Carga de datos
#
def load_data():

    import pandas as pd

    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    df = pd.read_csv(url, sep=";")

    y = df["quality"]
    x = df.copy()
    x.pop("quality")

    return x, y

In [29]:

    
#
# Particionamiento de datos
#
def make_train_test_split(x, y):

    from sklearn.model_selection import train_test_split

    (x_train, x_test, y_train, y_test) = train_test_split(
        x,
        y,
        test_size=0.25,
        random_state=0,
    )
    return x_train, x_test, y_train, y_test    
    
    

In [30]:
#
# Cálculo de metricas de evaluación
#
def eval_metrics(y_true, y_pred):

    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    return mse, mae, r2
    

In [31]:
#
# Reporte de métricas de evaluación
#
def report(estimator, mse, mae, r2):

    print(estimator, ":", sep="")
    print(f"  MSE: {mse}")
    print(f"  MAE: {mae}")
    print(f"  R2: {r2}")
    
    

In [32]:
    
    
#
# Almacenamiento del modelo
#
def save_best_estimator(estimator):

    import os
    import pickle

    with open("estimator.pickle", "wb") as file:
        pickle.dump(estimator, file)
        
        
        

In [33]:
#
# Carga del modelo
#
def load_best_estimator():

    import os
    import pickle

    if not os.path.exists("estimator.pickle"):
        return None
    with open("estimator.pickle", "rb") as file:
        estimator = pickle.load(file)

    return estimator
    
    
    
    

In [34]:
#
# Entrenamiento
#
def train_estimator(alpha=0.5, l1_ratio=0.5, verbose=1):

    from sklearn.linear_model import ElasticNet

    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)
    estimator = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=12345)
    estimator.fit(x_train, y_train)
    mse, mae, r2 = eval_metrics(y_test, y_pred=estimator.predict(x_test))
    if verbose > 0:
        report(estimator, mse, mae, r2)

    best_estimator = load_best_estimator()
    if best_estimator is None or estimator.score(x_test, y_test) > best_estimator.score(
        x_test, y_test
    ):
        best_estimator = estimator

    save_best_estimator(best_estimator)
    
    
    

In [35]:
def pipeline(alpha=0.5, l1_ratio=0.5, verbose=1):
    """
    Pipeline para entrenar el modelo y guardarlo.
    """
    train_estimator(alpha=alpha, l1_ratio=l1_ratio, verbose=verbose)
    print("Modelo entrenado y guardado en 'estimator.pickle'.")
    print("Para predecir, use el comando 'predict.py'.")  

In [36]:
alpha = [0.1, 0.2, 0.3, 0.4]
l1_ratio = [0.1, 0.2, 0.3, 0.4]
# Ejecutar el pipeline con diferentes combinaciones de alpha y l1_ratio

for i,j in  zip(alpha, l1_ratio):
    pipeline(alpha=i, l1_ratio=j, verbose=1)

ElasticNet(alpha=0.1, l1_ratio=0.1, random_state=12345):
  MSE: 0.4183271587407731
  MAE: 0.5055024368693067
  R2: 0.31555720466583137
Modelo entrenado y guardado en 'estimator.pickle'.
Para predecir, use el comando 'predict.py'.
ElasticNet(alpha=0.2, l1_ratio=0.2, random_state=12345):
  MSE: 0.43869119518947153
  MAE: 0.5236106762028768
  R2: 0.2822387414965034
Modelo entrenado y guardado en 'estimator.pickle'.
Para predecir, use el comando 'predict.py'.
ElasticNet(alpha=0.3, l1_ratio=0.3, random_state=12345):
  MSE: 0.4564369948553093
  MAE: 0.5381363876807477
  R2: 0.25320408650234194
Modelo entrenado y guardado en 'estimator.pickle'.
Para predecir, use el comando 'predict.py'.
ElasticNet(alpha=0.4, l1_ratio=0.4, random_state=12345):
  MSE: 0.48712574748446
  MAE: 0.5580061563349417
  R2: 0.20299291757407534
Modelo entrenado y guardado en 'estimator.pickle'.
Para predecir, use el comando 'predict.py'.


In [37]:
def check_estimator():

    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)
    estimator = load_best_estimator()
    mse, mae, r2 = eval_metrics(y_test, y_pred=estimator.predict(x_test))
    report(estimator, mse, mae, r2)


#
# Debe coincidir con el mejor modelo encontrado en la celdas anteriores
#
check_estimator()

ElasticNet(alpha=0.1, l1_ratio=0.1, random_state=12345):
  MSE: 0.4183271587407731
  MAE: 0.5055024368693067
  R2: 0.31555720466583137


In [38]:


def make_hyperparameters_search(alphas, l1_ratios):

    for alpha in alphas:
        for l1_ratio in l1_ratios:
            train_estimator(alpha=alpha, l1_ratio=l1_ratio, verbose=0)
            
            

In [39]:
    
import numpy as np

alphas = np.linspace(0.0001, 0.5, 10)
l1_ratios = np.linspace(0.0001, 0.5, 10)
make_hyperparameters_search(alphas, l1_ratios)
check_estimator()

ElasticNet(alpha=np.float64(0.0001), l1_ratio=np.float64(0.0001),
           random_state=12345):
  MSE: 0.40021745821413146
  MAE: 0.4848004855172136
  R2: 0.34518725328239785


In [40]:


def train_estimator(alphas, l1_ratios, n_splits=5, verbose=1):

    from sklearn.linear_model import ElasticNet
    from sklearn.model_selection import GridSearchCV

    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)

    # -------------------------------------------------------------------------
    # Búsqueda de parámetros con validación cruzada
    #
    estimator = GridSearchCV(
        estimator=ElasticNet(
            random_state=12345,
        ),
        param_grid={
            "alpha": alphas,
            "l1_ratio": l1_ratios,
        },
        cv=n_splits,
        refit=True,
        verbose=0,
        return_train_score=False,
    )
    # -------------------------------------------------------------------------

    estimator.fit(x_train, y_train)

    estimator = estimator.best_estimator_

    mse, mae, r2 = eval_metrics(y_test, y_pred=estimator.predict(x_test))
    if verbose > 0:
        report(estimator, mse, mae, r2)

    best_estimator = load_best_estimator()
    if best_estimator is None or estimator.score(x_test, y_test) > best_estimator.score(
        x_test, y_test
    ):
        best_estimator = estimator

    save_best_estimator(best_estimator)

In [41]:
train_estimator(alphas, l1_ratios, n_splits=5, verbose=1)

ElasticNet(alpha=np.float64(0.0001), l1_ratio=np.float64(0.0001),
           random_state=12345):
  MSE: 0.40021745821413146
  MAE: 0.4848004855172136
  R2: 0.34518725328239785


In [42]:
check_estimator()

ElasticNet(alpha=np.float64(0.0001), l1_ratio=np.float64(0.0001),
           random_state=12345):
  MSE: 0.40021745821413146
  MAE: 0.4848004855172136
  R2: 0.34518725328239785


In [43]:
save_best_estimator(check_estimator())

ElasticNet(alpha=np.float64(0.0001), l1_ratio=np.float64(0.0001),
           random_state=12345):
  MSE: 0.40021745821413146
  MAE: 0.4848004855172136
  R2: 0.34518725328239785
